In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

import torch

TORCH = torch.__version__.split('+')[0]

if torch.cuda.is_available():
    CUDA = 'cu' + torch.version.cuda.replace('.', '')
else:
    CUDA = 'cpu'

!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv \
    -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

!pip install torch-geometric

import torch_geometric

2.2.2+cu121
12.1
True
Looking in links: https://data.pyg.org/whl/torch-2.2.2+cu121.html

[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


/local/data/ipv577/projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch
import torch_geometric
import os
import numpy as np
import scipy.sparse
from scipy.sparse import coo_matrix, diags

print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

from torch_geometric.datasets import Planetoid

/local/data/ipv577/projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.2.2+cu121
12.1
True


In [2]:

data_root = os.path.join(os.path.dirname(os.getcwd()), "data")
name = "Cora"
dataset_path = os.path.join(data_root, "Planetoid", name)

if not os.path.exists(dataset_path):
    print(f"Dataset {name} not yet available. Download starts ...")
else:
    print(f"Dataset {name} already available, no download necessary.")

print(os.path.join(data_root, "Planetoid"))

data = Planetoid(root=os.path.join(data_root, "Planetoid"), name=name)[0]

Dataset Cora already available, no download necessary.
/local/data/ipv577/projects/gcn-kipf-welling-2017/data/Planetoid


In [3]:
print(f"num nodes: {data.num_nodes}")
print(f"num edges: {data.num_edges}")
print(data.x.size()[1])
num_targets = data.y.unique().size()[0]
print(num_targets)

data.num_nodes

num nodes: 2708
num edges: 10556
1433
7


2708

In [4]:
# prepare A and D non-normalized
rows = data.edge_index[0]
cols = data.edge_index[1]
values = np.ones(len(rows))

A = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(data.num_nodes, data.num_nodes), dtype=np.float32)
print(A)
degrees = A.sum(axis=1).A1
D = scipy.sparse.diags(degrees, dtype=np.float32)
print(D)

<COOrdinate sparse matrix of dtype 'float32'
	with 10556 stored elements and shape (2708, 2708)>
  Coords	Values
  (633, 0)	1.0
  (1862, 0)	1.0
  (2582, 0)	1.0
  (2, 1)	1.0
  (652, 1)	1.0
  (654, 1)	1.0
  (1, 2)	1.0
  (332, 2)	1.0
  (1454, 2)	1.0
  (1666, 2)	1.0
  (1986, 2)	1.0
  (2544, 3)	1.0
  (1016, 4)	1.0
  (1256, 4)	1.0
  (1761, 4)	1.0
  (2175, 4)	1.0
  (2176, 4)	1.0
  (1629, 5)	1.0
  (1659, 5)	1.0
  (2546, 5)	1.0
  (373, 6)	1.0
  (1042, 6)	1.0
  (1416, 6)	1.0
  (1602, 6)	1.0
  (208, 7)	1.0
  :	:
  (431, 2694)	1.0
  (2695, 2694)	1.0
  (431, 2695)	1.0
  (2694, 2695)	1.0
  (2615, 2696)	1.0
  (986, 2697)	1.0
  (1400, 2698)	1.0
  (1573, 2698)	1.0
  (2630, 2699)	1.0
  (1151, 2700)	1.0
  (44, 2701)	1.0
  (2624, 2701)	1.0
  (186, 2702)	1.0
  (1536, 2702)	1.0
  (1298, 2703)	1.0
  (641, 2704)	1.0
  (287, 2705)	1.0
  (165, 2706)	1.0
  (169, 2706)	1.0
  (1473, 2706)	1.0
  (2707, 2706)	1.0
  (165, 2707)	1.0
  (598, 2707)	1.0
  (1473, 2707)	1.0
  (2706, 2707)	1.0
<DIAgonal sparse matrix of dty

In [5]:
# cpmpute A_self, D_norm and A_hat
A_self = A + scipy.sparse.diags(np.ones(data.num_nodes), dtype=np.float32)

degrees = A_self.sum(axis=1).A1
D_norm = scipy.sparse.diags(degrees, dtype=np.float32)

A_hat = (D_norm.power(-0.5) @ A_self @ D_norm.power(-0.5)).tocoo()

print(f"A_hat = {A_hat}")
print(type(A_hat))

A_hat = <COOrdinate sparse matrix of dtype 'float32'
	with 13264 stored elements and shape (2708, 2708)>
  Coords	Values
  (0, 0)	0.25
  (0, 633)	0.25
  (0, 1862)	0.22360679507255554
  (0, 2582)	0.25
  (1, 1)	0.25
  (1, 2)	0.20412413775920868
  (1, 652)	0.28867512941360474
  (1, 654)	0.3535533845424652
  (2, 1)	0.20412413775920868
  (2, 2)	0.1666666567325592
  (2, 332)	0.1666666567325592
  (2, 1454)	0.28867512941360474
  (2, 1666)	0.15430335700511932
  (2, 1986)	0.05025188997387886
  (3, 3)	0.4999999701976776
  (3, 2544)	0.4999999701976776
  (4, 4)	0.1666666567325592
  (4, 1016)	0.1666666567325592
  (4, 1256)	0.13608276844024658
  (4, 1761)	0.14433756470680237
  (4, 2175)	0.1666666567325592
  (4, 2176)	0.13608276844024658
  (5, 5)	0.25
  (5, 1629)	0.25
  (5, 1659)	0.28867512941360474
  :	:
  (2699, 2699)	0.4999999701976776
  (2700, 1151)	0.40824827551841736
  (2700, 2700)	0.4999999701976776
  (2701, 44)	0.28867512941360474
  (2701, 2624)	0.3333333134651184
  (2701, 2701)	0.333333313465

In [6]:
# transform A_hat to tensor 

A_processed = torch.sparse_coo_tensor(A_hat.nonzero(), 
                        A_hat.data, 
                        size=A_hat.shape, 
                        dtype=torch.float32)
A_processed.dtype

/tmp/ipykernel_324861/3640817437.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  A_processed = torch.sparse_coo_tensor(A_hat.nonzero(),


torch.float32

In [11]:
# GCN architecture
# tbd

data.y.unique()

tensor([0, 1, 2, 3, 4, 5, 6])

In [22]:
data.test_mask.sum()

tensor(1000)